## Method 1: Using max and min function 

In [19]:
class DebtSimplifier:
    def __init__(self):
        self.debts = []

    def add_debt(self, borrower, lender, amount):
        self.debts.append((borrower, lender, amount))
        self.net_balance = {}
        for borrower, lender, amount in self.debts:
            self.net_balance[borrower] = self.net_balance.get(borrower, 0) - amount
            self.net_balance[lender] = self.net_balance.get(lender, 0) + amount
    
    def simplify_debts(self):
        net_balance_copy = self.net_balance.copy()
        simplified_transactions = []
        while True:
            max_creditor = max(net_balance_copy, key=net_balance_copy.get)
            max_debtor = min(net_balance_copy, key=net_balance_copy.get)

            if net_balance_copy[max_creditor] == 0:
                break

            transaction_amount = min(-net_balance_copy[max_debtor], net_balance_copy[max_creditor])
            simplified_transactions.append((max_debtor, max_creditor, transaction_amount))

            net_balance_copy[max_debtor] += transaction_amount
            net_balance_copy[max_creditor] -= transaction_amount

        return simplified_transactions

# Example usage:
simplifier = DebtSimplifier()
simplifier.add_debt("Alice", "Bob", 40)
simplifier.add_debt("Bob", "Charlie", 20)
simplifier.add_debt("Charlie", "David", 50)
simplifier.add_debt("Fred", "Bob", 10)
simplifier.add_debt("Fred", "Charlie", 30)
simplifier.add_debt("Fred", "David", 10)
simplifier.add_debt("Fred", "Gabe", 10)
simplifier.add_debt("Gabe", "Alice", 30)
simplifier.add_debt("Gabe", "Charlie", 10)

simplified_transactions = simplifier.simplify_debts()
print("The optimal transaction is")
for debtor, creditor, amount in simplified_transactions:
    print(f"{debtor} owes {creditor} ${amount}")


The optimal transaction is
Fred owes David $60
Gabe owes Bob $30
Alice owes Charlie $10


## Method 2: Using merge sort algorithm instead of min, max 

In [51]:
class DebtSimplifier:
    def __init__(self):
        self.debts = []

    def add_debt(self, borrower, lender, amount):
        self.debts.append((borrower, lender, amount))
        self.net_balance = {}
        for borrower, lender, amount in self.debts:
            self.net_balance[borrower] = self.net_balance.get(borrower, 0) - amount
            self.net_balance[lender] = self.net_balance.get(lender, 0) + amount
    def merge_sort(arr, compare_func=None):
        arr_temp = list(arr)
        n = len(arr_temp)    
    
        if n > 1: 
            mid = n // 2
            arr_temp_left = arr_temp[:mid] 
            arr_temp_right = arr_temp[mid:]
      
            arr_temp_left = merge_sort(arr_temp_left, compare_func)
            arr_temp_right = merge_sort(arr_temp_right, compare_func)
              
            i = j = k = 0
            n_left, n_right = len(arr_temp_left), len(arr_temp_right)
              
            while i < n_left and j < n_right: 
                if compare_func(arr_temp_left[i], arr_temp_right[j]):
                    arr_temp[k] = arr_temp_left[i] 
                    i += 1
                else: 
                    arr_temp[k] = arr_temp_right[j] 
                    j += 1
                k += 1
              
            while i < n_left: 
                arr_temp[k] = arr_temp_left[i] 
                i += 1
                k += 1
     
            while j < n_right: 
                arr_temp[k] = arr_temp_right[j] 
                j += 1
                k += 1
                
        return arr_temp

    def merge_sort_dict_by_value(dictionary):

        def compare_dict_items(item1, item2):

            return item1[1] < item2[1]
    
        items = list(dictionary.items())
        sorted_items = merge_sort(items, compare_func=compare_dict_items)
        return dict(sorted_items)
    
    def simplify_debts(self):
        net_balance_copy = self.net_balance.copy()
        simplified_transactions = []
        
        while True:
            sorted_net_balances = merge_sort_dict_by_value(net_balance_copy)
            max_creditor = list(sorted_net_balances.keys())[-1]
            max_debtor = list(sorted_net_balances.keys())[0]

            if net_balance_copy[max_creditor] == 0:
                break

            transaction_amount = min(-net_balance_copy[max_debtor], net_balance_copy[max_creditor])
            simplified_transactions.append((max_debtor, max_creditor, transaction_amount))

            net_balance_copy[max_debtor] += transaction_amount
            net_balance_copy[max_creditor] -= transaction_amount

        return simplified_transactions
            
        
        
simplifier = DebtSimplifier()
simplifier.add_debt("Alice", "Bob", 40)
simplifier.add_debt("Bob", "Charlie", 20)
simplifier.add_debt("Charlie", "David", 50)
simplifier.add_debt("Fred", "Bob", 10)
simplifier.add_debt("Fred", "Charlie", 30)
simplifier.add_debt("Fred", "David", 10)
simplifier.add_debt("Fred", "Gabe", 10)
simplifier.add_debt("Gabe", "Alice", 30)
simplifier.add_debt("Gabe", "Charlie", 10)

simplified_transactions = simplifier.simplify_debts()
print("The optimal transaction is")
for debtor, creditor, amount in simplified_transactions:
    print(f"{debtor} owes {creditor} ${amount}")
        
        
        
        

The optimal transaction is
Fred owes David $60
Gabe owes Bob $30
Alice owes Charlie $10
